##### **Notebook Parameters**
Update the parameters to match the environment:
- **minSku** - minimum SKU size to scale down to.
- **maxSku** - maximum SKU size to scale up to.
- **utilizationTolerance** - The utilization tolerance we want to stay under. This allows a buffer for any in-flight operations.
- **capacityName** - The Fabric capacity name that will be monitored and scaled.
- **subscriptionId** - The Azure subscription id that the Fabric capacity resides in.
- **metricsAppWorkspaceName** - Name of the workspace that the Capacity Metrics App semantic model is in.
- **metricsAppModelName** - Name of the Capacity Metrics App semantic model. The default app name is **Fabric Capacity Metrics**.

In [ ]:
minSku = 'F2' 
maxSku = 'F128' 
utilizationTolerance = 95
capacityName = ''
subscriptionId = '' 
metricsAppWorkspaceName = '' 
metricsAppModelName = 'Fabric Capacity Metrics' 

##### **Assign service principal credential information**
Update the **keyVaultEndpoint** to the Azure Key Vault url and the secret name values if the service principal credentials are stored there.\
These credential information can be hard coded for testing purposes and to get started.

In [ ]:
keyVaultEndpoint = ''

tenantId = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_tenantId')
clientId = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_clientId')
secret = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_clientSecret')

##### **Acquire Tokens and create the API headers**
We need to acquire two tokens:
- PBI audience so that we're able to use the PBI/Fabric APIs.
- Azure Management audience to scale the capacity within Azure.

In [ ]:
from azure.identity import ClientSecretCredential

api_pbi = 'https://analysis.windows.net/powerbi/api/.default'
api_azuremgmt = 'https://management.core.windows.net/.default'

auth = ClientSecretCredential(tenant_id=tenantId, client_id=clientId, client_secret=secret)
header_pbi = {'Authorization': f'Bearer {auth.get_token(api_pbi).token}', 'Content-type': 'application/json'}
header_azuremgmt = {'Authorization': f'Bearer {auth.get_token(api_azuremgmt).token}', 'Content-type': 'application/json'}

##### **Refresh the Fabric Capacity Metrics App semantic model**
Find the dataset id of the Fabric Capacity Metrics App and refresh the required imported tables.

In [ ]:
import requests, json, time

response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header_pbi)

workspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == metricsAppWorkspaceName][0]
print(f'{workspaceId = }')

response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets", headers=header_pbi)

datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == metricsAppModelName][0]
print(f'{datasetId = }')

tableList = [{"table": "Capacities"}
            ,{"table": "TimePoints"}
            ,{"table": "Items"}
            ]
body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes", headers=header_pbi, data=json.dumps(body))

refreshId = response.headers.get('RequestId')
print(f'{refreshId = }')

for attempt in range(12): 
    # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header_pbi)
    if response.json().get('value')[0].get('status') != 'Unknown':
        print(f'Refresh Complete')
        break
    else:
        print(f'Refreshing tables ...')
        time.sleep(5)

##### **Create a mapping for Fabric capacity sku and the allotted capacity units seconds (CUs)**
For each SKU size, build a dictionary for how many allotted capacity units by operation type (interactive/background) over a 24 hours period.

In [ ]:
skuDict = {f'F{2**_}': {"Interactive":(2**_)*(60*60), "Background": (2**_)*(60*60*24)} for _ in range(1,12)}
print(f'{skuDict = }')

##### **Get the current SKU of the capacity, the amount of allotted capacity seconds per 24 hours, and capacity id.**

In [ ]:
import requests

response = requests.get("https://api.fabric.microsoft.com/v1/capacities", headers=header_pbi)

currentSku = [capacity.get('sku') for capacity in response.json().get('value') if capacity.get('displayName') == capacityName][0]
print(f'{currentSku = }')
currentSkuCUTotalPerDay = skuDict.get(currentSku).get("Background")
print(f'{currentSkuCUTotalPerDay = }')
capacityId = [capacity.get('id') for capacity in response.json().get('value') if capacity.get('displayName') == capacityName][0].upper()
print(f'{capacityId = }')

##### **Query the Fabric Capacity Metrics App model**
Execute a DAX query against the Fabric Capacity Metrics App to get the amount of capacity seconds (CUs) by operation type (interaction/background) and billing type that have occurred over the last 24 hours.

In [ ]:
import requests, math

body = {
  "queries": [
    {
      "query": f"""
        DEFINE
            MPARAMETER 'CapacityID' = "{capacityId}"

            VAR __DS0FilterTable2 = TREATAS({{"{capacityId}"}}, 'Capacities'[capacityId])
            
            VAR __DS0FilterTable3 = 
                      FILTER(
                        KEEPFILTERS(VALUES('TimePoints'[TimePoint])),
                        'TimePoints'[TimePoint] >= NOW() - 1
                      )
                      
            VAR __DS0Core = 
              SELECTCOLUMNS(
                SUMMARIZECOLUMNS(
                'Capacities'[Capacity Name],
                Items[Billable type],
                  __DS0FilterTable2,
                  __DS0FilterTable3,
                  "SumInteractive", SUM('CUDetail'[Interactive]),
                  "SumBackground", SUM('CUDetail'[Background]),
                  "SumCUs", SUM('CUDetail'[CUs])
                ),
                "CapacityName", 'Capacities'[Capacity Name],
                "BillType", Items[Billable type],
                "SumInteractive", [SumInteractive],
                "SumBackground", [SumBackground],
                "SumCUs", [SumCUs]
              )

          EVALUATE
            __DS0Core
    """
    }
  ]
}

response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header_pbi, json=body )

totalConsumedCULast24HoursInteractive = 0
totalConsumedCULast24HoursBackground = 0
for results in response.json().get('results'):
    for table in results.get('tables'):
        for row in table.get('rows'):
          totalConsumedCULast24HoursInteractive += row.get('[SumInteractive]')
          totalConsumedCULast24HoursBackground += row.get('[SumBackground]')

totalConsumedCULast24Hours = math.ceil(totalConsumedCULast24HoursInteractive + totalConsumedCULast24HoursBackground)
print(f'{totalConsumedCULast24Hours = }')

##### **Calculate the optimal SKU size**
Apply logic to determine if the capacity should be scaled and if it needs to scale, what capacity should it scale to based on the current capacity, consumption over the last 24 hours, and the defined utilization tolerance to be within the defined min/max SKU.

In [ ]:
import math

utilizationTolerancePercentage = utilizationTolerance/100

print(f'{totalConsumedCULast24Hours = }')
print(f'{utilizationTolerancePercentage = }')
print(f'{currentSkuCUTotalPerDay = }')

skuNeeded = [(sku, cuDict.get("Background"), totalConsumedCULast24Hours, math.ceil(cuDict.get("Background")*utilizationTolerancePercentage)) for sku, cuDict in skuDict.items() if math.ceil(cuDict.get("Background")*utilizationTolerancePercentage) >= totalConsumedCULast24Hours][0]
print(f'{skuNeeded = }')

scaleSku = ''
if int(skuNeeded[0].replace('F', '')) < int(minSku.replace('F', '')):
    scaleSku = minSku
elif int(skuNeeded[0].replace('F', '')) > int(maxSku.replace('F', '')):
    scaleSku = maxSku
else:
    scaleSku = skuNeeded[0]

print(f'{scaleSku = }')

##### **Perform the scaling operation within Azure**
If the current SKU of the capacity is different than the calculated optimal SKU, then perform the scaling operation.

In [ ]:
import requests, json

# Validation to check if the sku to scale to is different than the current sku
if scaleSku != currentSku:
    print(f'\nScaling from {currentSku} to {scaleSku}')

    response = requests.get(f'https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.Fabric/capacities?api-version=2022-07-01-preview', headers=header_azuremgmt)
    responseList = response.json().get('value')
    resourceGroupName = [resource.get('id') for resource in responseList if resource.get('name') == capacityName][0].split("resourceGroups/")[-1].split("/")[0]

    url = f'https://management.azure.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.Fabric/capacities/{capacityName}?api-version=2022-07-01-preview'
    body = {"sku": {"name": f"{scaleSku}", "tier": "Fabric"}}
            
    response = requests.patch(url, headers=header_azuremgmt, data=json.dumps(body))
    print(response, response.text)

else:
    print(f'The current SKU of {currentSku} is the optimal SKU based off the utilization tolerance of {utilizationTolerance}%, minSku {minSku}, maxSku {maxSku}, and CU consumption over the last 24 hours {totalConsumedCULast24Hours:,}.')